In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Define the year and load the data
YEAR = 2024
url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
data = pd.read_csv(url, compression='gzip', low_memory=False)

playoff_data = data[data['week'].isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19, 20, 21])]
mahomes_throws = playoff_data[
    (playoff_data['passer_player_name'] == 'P.Mahomes') &
    (~playoff_data['air_yards'].isna()) &
    (~playoff_data['pass_location'].isna()) & 
    (~playoff_data['epa'].isna()) 
]

mahomes_throws['air_yards_bucket'] = pd.cut(
    mahomes_throws['air_yards'], bins=range(-10, 61, 10), right=False, include_lowest=True
)

heatmap_data = mahomes_throws.groupby(['pass_location', 'air_yards_bucket']).agg(
    attempts=('air_yards', 'count'),
    completions=('complete_pass', 'sum'),
    total_epa=('epa', 'sum')
).reset_index()

heatmap_data['epa_per_play'] = heatmap_data['total_epa'] / heatmap_data['attempts']
heatmap_data['epa_per_play'].fillna(0, inplace=True) 

heatmap_data['completions_attempts'] = (
    heatmap_data['completions'].astype(int).astype(str) + "/" + 
    heatmap_data['attempts'].astype(int).astype(str)
)

heatmap_data['air_yards_bucket'] = heatmap_data['air_yards_bucket'].astype(str)

y_axis_labels = {
    "[-10, 0)": "<0",
    "[0, 10)": "0-10",
    "[10, 20)": "10-20",
    "[20, 30)": "20-30",
    "[30, 40)": "30-40",
    "[40, 50)": "40+"
}

heatmap_data = heatmap_data[heatmap_data['air_yards_bucket'] != "[50, 60)"]

heatmap_data['air_yards_bucket'] = heatmap_data['air_yards_bucket'].map(y_axis_labels)

pivot_text = heatmap_data.pivot(index='air_yards_bucket', columns='pass_location', values='completions_attempts')
pivot_colors = heatmap_data.pivot(index='air_yards_bucket', columns='pass_location', values='epa_per_play')

pivot_text = pivot_text.reindex(index=["<0", "0-10", "10-20", "20-30", "30-40", "40+"])
pivot_colors = pivot_colors.reindex(index=["<0", "0-10", "10-20", "20-30", "30-40", "40+"])

fig = go.Figure(data=go.Heatmap(
    z=pivot_colors, 
    x=pivot_colors.columns,
    y=pivot_colors.index,
    colorscale="viridis",  
    zmin=-1,
    zmax=1,
    colorbar_title="EPA/Play"
))

for i, y_val in enumerate(pivot_text.index):
    for j, x_val in enumerate(pivot_text.columns):
        text = pivot_text.loc[y_val, x_val]
        epa_value = pivot_colors.loc[y_val, x_val]  
        
        if pd.notna(text):  
            font_color = "white" if epa_value < 0.875 else "black"  
            fig.add_annotation(
                x=x_val, y=y_val, text=text,
                showarrow=False, font=dict(size=14, color=font_color),
                align="center"
            )

fig.add_layout_image(
    dict(
        source="1.png",  
        x=1.19, y=1.19, 
        xref="paper", yref="paper",
        sizex=0.15, sizey=0.15,  
        xanchor="right", yanchor="top",
        layer="above"
    )
)

# Add footer
fig.add_annotation(
    text="Ray Carpenter | @array_carpenter | Data: NFLVerse | TheSpade.substack.com",
    xref="paper", yref="paper",
    x=0.05, y=-0.1,  
    showarrow=False,
    font=dict(size=12, color="gray")
)

# Adjust layout
fig.update_layout(
    title="Patrick Mahomes 2024 Playoff Passes by Air Yards & Direction",
    yaxis_title="Air Yards (10-Yard Buckets)",
    xaxis=dict(tickmode='array', tickvals=list(pivot_text.columns)),  
    yaxis=dict(tickmode='array', tickvals=list(pivot_text.index)),  
    autosize=False,
    width=700,  
    height=700  
)

fig.show()


C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\324265266.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\324265266.py:24: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\324265266.py:32: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object

In [ ]:
import pandas as pd
import plotly.graph_objects as go

YEAR = 2024
url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{YEAR}.csv.gz'
data = pd.read_csv(url, compression='gzip', low_memory=False)

playoff_data = data[data['week'].isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])]
mahomes_throws = playoff_data[
    (playoff_data['passer_player_name'] == 'P.Mahomes') &
    (~playoff_data['air_yards'].isna()) &
    (~playoff_data['pass_location'].isna()) & 
    (~playoff_data['epa'].isna()) 
]

mahomes_throws['air_yards_bucket'] = pd.cut(
    mahomes_throws['air_yards'], bins=range(-10, 61, 10), right=False, include_lowest=True
)

heatmap_data = mahomes_throws.groupby(['pass_location', 'air_yards_bucket']).agg(
    attempts=('air_yards', 'count'),
    completions=('complete_pass', 'sum'),
    total_epa=('epa', 'sum')
).reset_index()

heatmap_data['epa_per_play'] = heatmap_data['total_epa'] / heatmap_data['attempts']
heatmap_data['epa_per_play'].fillna(0, inplace=True) 

heatmap_data['completions_attempts'] = (
    heatmap_data['completions'].astype(int).astype(str) + "/" + 
    heatmap_data['attempts'].astype(int).astype(str)
)

heatmap_data['air_yards_bucket'] = heatmap_data['air_yards_bucket'].astype(str)

y_axis_labels = {
    "[-10, 0)": "<0",
    "[0, 10)": "0-10",
    "[10, 20)": "10-20",
    "[20, 30)": "20-30",
    "[30, 40)": "30-40",
    "[40, 50)": "40+"
}

heatmap_data = heatmap_data[heatmap_data['air_yards_bucket'] != "[50, 60)"]

heatmap_data['air_yards_bucket'] = heatmap_data['air_yards_bucket'].map(y_axis_labels)

pivot_text = heatmap_data.pivot(index='air_yards_bucket', columns='pass_location', values='completions_attempts')
pivot_colors = heatmap_data.pivot(index='air_yards_bucket', columns='pass_location', values='epa_per_play')

pivot_text = pivot_text.reindex(index=["<0", "0-10", "10-20", "20-30", "30-40", "40+"])
pivot_colors = pivot_colors.reindex(index=["<0", "0-10", "10-20", "20-30", "30-40", "40+"])

fig = go.Figure(data=go.Heatmap(
    z=pivot_colors,  
    x=pivot_colors.columns,
    y=pivot_colors.index,
    colorscale="viridis", 
    zmin=-1, 
    zmax=1,
    colorbar_title="EPA/Play"
))

for i, y_val in enumerate(pivot_text.index):
    for j, x_val in enumerate(pivot_text.columns):
        text = pivot_text.loc[y_val, x_val]
        epa_value = pivot_colors.loc[y_val, x_val] 
        
        if pd.notna(text): 
            font_color = "white" if epa_value < 0.875 else "black" 
            fig.add_annotation(
                x=x_val, y=y_val, text=text,
                showarrow=False, font=dict(size=14, color=font_color),
                align="center"
            )

fig.add_layout_image(
    dict(
        source="1.png",  
        x=1.19, y=1.19,  
        xref="paper", yref="paper",
        sizex=0.15, sizey=0.15,  
        xanchor="right", yanchor="top",
        layer="above"
    )
)

# Add footer
fig.add_annotation(
    text="Ray Carpenter | @array_carpenter | Data: NFLVerse | TheSpade.substack.com",
    xref="paper", yref="paper",
    x=0.05, y=-0.1,  
    showarrow=False,
    font=dict(size=12, color="gray")
)

# Adjust layout
fig.update_layout(
    title="Patrick Mahomes 2024 Reg+Postseason Passes by Air Yards & Direction",
    yaxis_title="Air Yards (10-Yard Buckets)",
    xaxis=dict(tickmode='array', tickvals=list(pivot_text.columns)),  
    yaxis=dict(tickmode='array', tickvals=list(pivot_text.index)),  
    autosize=False,
    width=700,  
    height=700 
)

fig.show()


C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\2553254686.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\2553254686.py:24: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\RaymondCarpenter\AppData\Local\Temp\ipykernel_19724\2553254686.py:32: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate obj

In [119]:
import pandas as pd
import plotly.express as px


def load_data(year):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv.gz'
    return pd.read_csv(url, compression='gzip', low_memory=False)


data_2024 = load_data(2024)
data_2023 = load_data(2023)
data_2022 = load_data(2022)
data_2021 = load_data(2021)


columns_to_include = [
    'penalty', 'first_down_penalty', 'penalty_team', 'penalty_player_id',
    'penalty_player_name', 'penalty_yards', 'penalty_type', 
    'desc', 'home_wp', 'away_wp'
]


phi_games = pd.concat([
    data_2024[(data_2024['week'].isin([1, 18])) & ((data_2024['home_team'] == 'PHI') | (data_2024['away_team'] == 'PHI'))],
    data_2023[(data_2023['week'].isin([4, 17])) & ((data_2023['home_team'] == 'PHI') | (data_2023['away_team'] == 'PHI'))],
    data_2022[(data_2022['week'] == 4) & ((data_2022['home_team'] == 'PHI') | (data_2022['away_team'] == 'PHI'))],
    data_2021[(data_2021['week'].isin([12,18])) & ((data_2021['home_team'] == 'PHI') | (data_2021['away_team'] == 'PHI'))]
])

penalties = phi_games[phi_games['penalty'] == 1][columns_to_include]

phi_penalties = penalties[penalties['penalty_team'] == 'PHI']

penalty_counts = phi_penalties.groupby('penalty_type').size().reset_index(name='count')


fig = px.bar(
    penalty_counts,
    x='penalty_type',
    y='count',
    title='Philadelphia Eagles Penalties by Type in Ron Torbert Games (2021-2024)',
    color_discrete_sequence=['#004C54'],
    width=700 
)

fig.update_traces(text=penalty_counts['count'], textposition='outside')


fig.update_layout(
    xaxis_tickangle=-45, 
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(range=[0, penalty_counts['count'].max() + 3], dtick=2),
    xaxis_title='',
    yaxis_title='',  
    

    annotations=[
        dict(
            text='n=7 | Data source: NFLverse | By Ray Carpenter | TheSpade.substack.com',
            showarrow=False,
            x=0.5,
            y=-1,
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='bottom',
            font=dict(size=12, color='gray')
        )
    ],

    images=[
        dict(
            source="C:/Users/RaymondCarpenter/Documents/GitHub/nfl_throwing_scorecard/2.png",
            xref="paper", yref="paper",
            x=1.05, y=-3, 
            sizex=1, sizey=1, 
            xanchor="right", yanchor="bottom",
            layer="above",
            opacity=1 
        )
    ]
)

fig.show()


In [118]:
import pandas as pd
import plotly.express as px

# Function to load data for a specific year
def load_data(year):
    url = f'https://github.com/nflverse/nflverse-data/releases/download/pbp/play_by_play_{year}.csv.gz'
    return pd.read_csv(url, compression='gzip', low_memory=False)

# Load data for required years
data_2024 = load_data(2024)
data_2023 = load_data(2023)
data_2022 = load_data(2022)
data_2021 = load_data(2021)

# Columns to include
columns_to_include = [
    'penalty', 'first_down_penalty', 'penalty_team', 'penalty_player_id',
    'penalty_player_name', 'penalty_yards', 'penalty_type', 
    'desc', 'home_wp', 'away_wp'
]

# Filter for KC's games in specified weeks
kc_games = pd.concat([
    data_2022[(data_2022['week'] == 21) & ((data_2022['home_team'] == 'KC') | (data_2022['away_team'] == 'KC'))],
    data_2021[(data_2021['week'] == 17) & ((data_2021['home_team'] == 'KC') | (data_2021['away_team'] == 'KC'))]
])

penalties = kc_games[kc_games['penalty'] == 1][columns_to_include]

kc_penalties = penalties[penalties['penalty_team'] == 'KC']

penalty_counts = kc_penalties.groupby('penalty_type').size().reset_index(name='count')

fig = px.bar(
    penalty_counts,
    x='penalty_type',
    y='count',
    title='Kansas City Chiefs Penalties by Type in Ron Torbert Games (2021-2024)',
    color_discrete_sequence=['#E31837'], 
    width=700
)

fig.update_traces(text=penalty_counts['count'], textposition='outside')

fig.update_layout(
    xaxis_tickangle=-45,  
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(range=[0, penalty_counts['count'].max() + 3], dtick=2),
    xaxis_title='',  
    yaxis_title='', 
    
    annotations=[
        dict(
            text='n=2 | Data source: NFLverse | By Ray Carpenter | TheSpade.substack.com',
            showarrow=False,
            x=0.5,
            y=-.8,  # Move the footer further down
            xref='paper',
            yref='paper',
            xanchor='center',
            yanchor='bottom',
            font=dict(size=12, color='gray')
        )
    ],

    images=[
        dict(
            source="C:/Users/RaymondCarpenter/Documents/GitHub/nfl_throwing_scorecard/2.png",
            xref="paper", yref="paper",
            x=1.05, y=-3, 
            sizex=1, sizey=1,
            xanchor="right", yanchor="bottom",
            layer="above",
            opacity=1 
        )
    ]
)

fig.show()
